<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Moissonnage_de_donnees_sur_la_Toile.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

# Moissonnage de données sur la Toile

## Les marées des sept prochains jours à Tadoussac 

<p>Vous planifiez une sortie en kayak de mer à Tadoussac qui est située sur le fleuve Saint-Laurent à l'embouchure du Saguenay. Vous allez moissonner les informations concernant les marées pour les sept prochains jours à compter du 21 septembre 2021. Pour cela, vous utiliserez un formulaire disponible sur le site de Pêches et Océans Canada.</p> 

<p>Pour simuler les interactions que fait un humain avec le site web, vous utiliserez la bibliothèque python <a href="https://selenium-python.readthedocs.io/" target='_blank'>Selenium</a>. L'extraction des sections pertinentes de la page web des résultats sera faite avec l'outil python <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/" target='_blank'>Beautiful Soup</a>. Enfin, des <a href="https://docs.python.org/fr/3/howto/regex.html" target='_blank'>expressions régulières<sup>1</sup></a> seront utilisées pour extraire les informations détaillées.</p>
<hr/>
<span style="font-size:80%"><sup>1</sup><b>Note - pratique: </b><a href="https://regex101.com/" target='_blank'>regex101.com</a> est un excellent site pour pratiquer et metre au point des expressions régulières.</span>

## Importation des bibliothèques Python

In [ ]:
# -*- coding: utf-8 -*-

import requests, csv, os, time
# sudo pip3 install bs4
from bs4 import BeautifulSoup
# Installer ChromeDriver sur mac OS
# brew install chromedriver
# sudo pip3 install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

print("Bibliothèques Python importées")

## Analyse des points d'entrée du formulaire avec la fonction d'inspection de votre fureteur
<ul>
    <ul>
        <li>Ouvrir la page du <a href="https://www.marees.gc.ca/fra/station?sid=2985" target='_blank'>formulaire</a> sur le site de Pêches et Océans Canada;</li>
        <li>Reconstituer la séquence des interactions avec le formulaire (clics, menus et saisie clavier);</li>
        <li>Obtenir les identifiants des boutons, des menus, des champs de saisie;</li>
    </ul>
</ul>   

## Programmation d'un script d'interaction Selenium avec le formulaire

In [ ]:
# Instanciation d'un pilote pour le fureteur Chrome 
pilote_chrome = webdriver.Chrome("/usr/local/bin/chromedriver")

# Obtenir la page du formulaire de Pêches et Océans
pilote_chrome.get("https://www.marees.gc.ca/fra/station?sid=2985")

date_requise = "2021/09/21"
saisie_date = pilote_chrome.find_element_by_id("date")
saisie_date.clear()
saisie_date.send_keys(date_requise, Keys.RETURN)

municipalite_requise = "Tadoussac"
# Obtenir le menu déroulant des municipalités
menu_deroulant_localites = pilote_chrome.find_element_by_id("sid")
# Obtenir la liste des municipalités
liste_municipalites = menu_deroulant_localites.find_elements_by_tag_name('option')
# Choisir la municipalité requise
for municipalite in liste_municipalites:
    # print(municipalite.text)
    if (municipalite.text == municipalite_requise):
        municipalite.click()

format_requis = "texte"
# Obtenir le menu déroulant des formats de sortie
menu_deroulant_formats = pilote_chrome.find_element_by_id("pres")
# Obtenir la liste des formats de sortie
liste_formats = menu_deroulant_formats.find_elements_by_tag_name('option')
# Choisir le format requis
for format_type in liste_formats:
    # print(format_type.text)
    if (format_type.text == format_requis):
        format_type.click()

# Activer la soumission du formulaire     
bouton_soumission = pilote_chrome.find_element_by_css_selector("input.button.button-accent")       
bouton_soumission.click()

# Obtenir la page web des résultats
page_reponse = pilote_chrome.page_source

print("Script Selenium exécuté, formulaire soumis...")

## Analyse de la page de réponse retournée avec BeautifulSoup

In [ ]:
# Analyser la page de réponse retournée avec BeautifulSoup
dom_page_resultats = BeautifulSoup(page_reponse,"html.parser") 

# Afficher la page web retournée
print(dom_page_resultats.prettify())

## Extraction des sections pertinentes de la page de résultats avec Beautiful Soup;

In [ ]:
# Extraire de la page les sections pertinentes
# cela implique un examen du contenu de la page et
# une connaissance des possibilités de BeautifulSoup
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

# Extraire les paramètres de la requête de recherche d'information
parametres_bruts = dom_page_resultats.find("div", {"class":"stationTextHeader"})
# print("Paramètres bruts:")
parametres_bruts = parametres_bruts.getText().strip()

# Extraire les données de marée 
donnees_marees_brutes = dom_page_resultats.find("div", {"class":"stationTextData"})
# print("Données marées brutes:\n",donnees_marees_brutes)
donnees_marees_brutes = donnees_marees_brutes.getText().strip()
donnees_marees_brutes_liste = donnees_marees_brutes.split('\n')

print("Sections pertinentes de la page de résultats isolées")

## Extraction des informations détaillées avec des expressions régulières;

In [ ]:
# Extraction des attributs au moyen d'une expression régulière
# la mise au point de l'expression régulière se fait avec https://regex101.com/
import re
FORME1 = re.compile(r'# Station : (\W*\w*)\s\((\d*)\)\s*#\sfuseau horaire : (\w*)[\W*\w*\s*]*')
formes_reconnues = re.match(FORME1,parametres_bruts)
if formes_reconnues:
    localite = formes_reconnues.group(1)
    code_station = formes_reconnues.group(2)
    fuseau_horaire = formes_reconnues.group(3)
    # print(localite+'\t'+code_station+'\t'+fuseau_horaire)

# Extraction des attributs au moyen d'une expression régulière
# la mise au point de l'expression régulière se fait avec https://regex101.com/
import re
FORME2 = re.compile(r'\s*(\d*-\d*-\d*);(\d*:\d*:\d*);(\d*.\d*)\(m\);\d*.\d*\(ft\)$')
donnees_marees_liste = []
for data_element in donnees_marees_brutes_liste:
    formes_reconnues = re.match(FORME2,data_element)
    if formes_reconnues:
        date = formes_reconnues.group(1)
        heure = formes_reconnues.group(2)
        hauteur = formes_reconnues.group(3)
        donnees_chaine = date+'\t'+heure+'\t'+hauteur
        # print(donnees_chaine)
        donnees_marees_liste.append(donnees_chaine)
        
print("Informations détaillées sur les marées extraites")

## Sauvegarde des données de marées dans le fichier `marees_donnees.csv`

In [ ]:
# Sauvegarder les données dans un fichier .csv
chemin_fichier_sortie = './'
nom_fichier_sortie = "marees_donnees.csv"
with open(chemin_fichier_sortie+nom_fichier_sortie,'w') as fichier_sortie:
    # Écriture de l'entête du fichier listant les différents attributs
    fichier_sortie.write('localite\tstation\tfuseau_horaire\tdate\theure\thauteur_m\n')
    for donnees_marees in donnees_marees_liste:
        # Écriture des données sur la marée dans le fichier
        fichier_sortie.write(localite+'\t'+code_station+'\t'+fuseau_horaire+'\t'+donnees_marees+'\n')
        
print("Données de marées sauvegardées dans le fichier `marees_donnees.csv`")

## Test de lecture du fichier de données

In [ ]:
import pandas as pd

donnees_marees_df = pd.read_csv(chemin_fichier_sortie+nom_fichier_sortie,delimiter='\t')
donnees_marees_df